# Lab 1: ReActエージェントをゼロから構築する

## ReActパターン

このセクションでは、ReAct（Reasoning and Acting：推論と行動）パターンを使用してAIエージェントを構築します。この概念に馴染みがなくても心配いりません—ステップバイステップで説明していきます。

ReActパターンは、人間の認知パターンを模倣したAIの問題解決プロセスを構造化するフレームワークです：

1. **推論（Reason）**：現状について考える
2. **決定（Decide）**：取るべき行動を決める
3. **観察（Observe）**：その行動の結果を観察する
4. **繰り返し（Repeat）**：タスクが完了するまで繰り返す

この概念を説明するために、経験豊富なソフトウェアエンジニアが複雑なシステムのデバッグにどのようにアプローチするかを考えてみましょう：

1. **推論**：エラーログとシステム状態を分析する（例：「データベース接続がタイムアウトしている」）
2. **行動**：診断アクションを実行する（例：「データベース接続テストを実行する」）
3. **観察**：診断結果を調べる（例：「テストで高いレイテンシーが示されている」）
4. **繰り返し**：問題が解決するまでこのプロセスを続け、次にネットワーク設定を確認するなど

私たちのAIエージェントも同様の方法論で問題に取り組みます。このエージェントを開発する際、AIモデル（推論と決定を行う「脳」）と私たちのPythonコード（環境と相互作用しプロセスフローを管理する「体」）の間の分業に注目してください。

このノートブックは、[Simon Willisonによる以下のノートブック](https://til.simonwillison.net/llms/python-react-pattern)に基づいています。

## 環境のセットアップ

まず、必要なライブラリをインポートし、環境を構成しましょう。

### Bedrockクライアントの初期化

Amazon Bedrockを通じてClaudeモデルと通信するには、クライアント接続を確立する必要があります。このクライアントは、私たちのコードがAIモデルにリクエストを送信し、応答を受け取ることを可能にするAPIゲートウェイのようなものと考えてください。

私たちは`boto3`ライブラリを使用します。これはAmazon Web Services（AWS）のPython用SDKです。AWSに馴染みのない方にとって、`boto3`はPythonがBedrockを含む様々なAWSサービスと対話することを容易にする包括的なツールキットと考えることができます。

`boto3`をAWS認証情報で構成する詳細な手順については、[AWSドキュメント](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html)を参照してください。

本番環境では、安全なAWS認証情報管理を実装することになります。このセクションの目的では、認証情報が環境内で事前に構成されていることを前提とします。

In [1]:
from dotenv import load_dotenv
import os
import sys
import boto3
import re
from botocore.config import Config
import warnings

warnings.filterwarnings("ignore")
import logging

# import local modules
dir_current = os.path.abspath("")
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

# Set basic configs
logger = utils.set_logger()
pp = utils.set_pretty_printer()

# Load environment variables from .env file
_ = load_dotenv("../.env")
aws_region = os.getenv("AWS_REGION")

# Set bedrock configs
bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Create a bedrock runtime client in your aws region.
# If you do not have the AWS CLI profile setup, you can authenticate with aws access key, secret and session token.
# For more details check https://docs.aws.amazon.com/cli/v1/userguide/cli-authentication-short-term.html
bedrock_rt = boto3.client(
    "bedrock-runtime",
    region_name=aws_region,
    config=bedrock_config,
)

まず、いくつかの推論パラメータを定義し、`boto3`を通じてAmazon Bedrockへの接続をテストします。

In [2]:
# Set inference parameters
temperature = 0.0
top_k = 200
inference_config = {"temperature": temperature}

additional_model_fields = {"top_k": top_k}
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
system_prompts = [{"text": "You are a helpful agent."}]
message_1 = {"role": "user", "content": [{"text": "こんにちは"}]}

# Instantiate messages list
messages = []
messages.append(message_1)

# Send the message.
response = bedrock_rt.converse(
    modelId=model_id,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config,
    additionalModelRequestFields=additional_model_fields,
)

pp.pprint(response)
print("\n\n")
pp.pprint(response["output"]["message"]["content"][0]["text"])

{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '382',
                                         'content-type': 'application/json',
                                         'date': 'Wed, 09 Jul 2025 16:54:32 GMT',
                                         'x-amzn-requestid': '04580d28-3886-4fea-8b11-4a137e3435d9'},
                        'HTTPStatusCode': 200,
                        'RequestId': '04580d28-3886-4fea-8b11-4a137e3435d9',
                        'RetryAttempts': 0},
  'metrics': {'latencyMs': 2499},
  'output': { 'message': { 'content': [ { 'text': 'こんにちは!私は人工知能のAssistantです。どのようなことでお手伝いできましょうか?質問や課題などがあれば、喜んでサポートさせていただきます。'}],
                           'role': 'assistant'}},
  'stopReason': 'end_turn',
  'usage': {'inputTokens': 18, 'outputTokens': 63, 'totalTokens': 81}}



'こんにちは!私は人工知能のAssistantです。どのようなことでお手伝いできましょうか?質問や課題などがあれば、喜んでサポートさせていただきます。'


## エージェントクラスの設計

Bedrockクライアントのセットアップが完了したので、次にAgentクラスを作成します。このクラスは、Claudeモデルとの対話ロジックと会話状態の維持をカプセル化する、AIエージェントの中核として機能します。

私たちのエージェントが実装するReActパターンは、主に3つのステップで構成されています：

1. **推論（Thought）**：エージェントは現状を評価し、計画を立てます。例えば、「2つの犬種の総重量を計算するには、それぞれの個別の重量を調べて合計する必要がある」といった具合です。

2. **行動（Action）**：推論に基づいて、エージェントは適切な行動を選択します。例えば、「ボーダー・コリーの平均体重を調べる」などです。

3. **観察（Observation）**：エージェントは行動からのフィードバックを処理します。この場合、「ボーダー・コリーの平均体重は30〜55ポンドである」といった情報になります。

このパターンにより、エージェントは複雑なタスクを管理可能なステップに分解し、新しい情報に基づいて戦略を適応させることができます。

私たちのAgentクラスは、会話履歴（`self.messages`）を維持し、Claudeモデルと対話するためのメソッド（`__call__`と`execute`）を提供することで、このパターンを実装します。

In [3]:
class Agent:
    def __init__(self, system=""):
        # システムプロンプトを初期化（エージェントの基本的な指示を設定）
        self.system = system
        # 会話履歴を保存するリストを初期化
        self.messages = []
        # システムプロンプトが提供された場合、適切な形式に変換
        if self.system:
            self.system = [{"text": self.system}]
        # AWS Bedrockのクライアントを初期化
        self.bedrock_client = boto3.client(service_name="bedrock-runtime")

    def __call__(self, message):
        # ユーザーメッセージを会話履歴に追加
        self.messages.append({"role": "user", "content": [{"text": message}]})
        # モデルから応答を取得
        result = self.execute()
        # アシスタントの応答を会話履歴に追加
        self.messages.append({"role": "assistant", "content": [{"text": result}]})
        # 応答を返す
        return result

    def execute(self):
        # 推論設定を定義
        inference_config = {
            "temperature": 0.0,  # 決定論的な応答を得るため温度を0に設定
            "stopSequences": ["<PAUSE>"],  # 後で探索する重要な停止シーケンス
        }
        # モデル固有の追加設定
        additional_model_fields = {"top_k": 200}  # 考慮する最上位トークンの数
        
        # Bedrockを通じてClaudeモデルに問い合わせ
        response = self.bedrock_client.converse(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",  # 使用するモデルのID
            messages=self.messages,  # 会話履歴
            system=self.system,  # システムプロンプト
            inferenceConfig=inference_config,  # 推論設定
            additionalModelRequestFields=additional_model_fields,  # 追加モデル設定
        )
        # モデルからの応答テキストを抽出して返す
        return response["output"]["message"]["content"][0]["text"]


## プロンプトの作成

プロンプトはAIモデルへの指示セットとして機能し、その動作と利用可能なアクションを定義する上で重要です。

私たちの実装では、モデルに以下のように指示しています：

- ReActパターン（思考、行動、観察のサイクル）に従う
- 各ステップに特定の形式を使用する（例：思考を「Thought:」で始める）
- 提供されたアクションのみを使用する（この場合、計算機と犬の体重検索機能）

また、期待される応答形式を示すためのサンプル対話も含めています。これは、複雑なフォームに記入してもらう前に、記入済みのテンプレートを提供するようなものです。

プロンプト、エージェントクラス、および推論パラメータからわかるように、モデルが`<PAUSE>`トークンを予測した後に生成を停止するよう要求しています。しかし、安全を期すために`<PAUSE>`を`stopSequences`に追加し、その後のトークン生成が確実に終了するようにしています！

In [4]:
prompt = """あなたは「思考（Thought）」、「行動（Action）」、「<PAUSE>」、「観察（Observation）」のループで実行されます。
ループの最後に「回答（Answer）」を出力します。

「Thought」を使って、あなたが質問されたことについての考えを説明してください。
「Action」を使って、利用可能なアクションのいずれかを実行し、その後PAUSEを返します。
「Observation」はそれらのアクションを実行した結果になります。

あなたが利用できるアクションは以下の通りです：

calculate:
例：calculate: 4 * 7 / 3
計算を実行して数値を返します - Pythonを使用するので、必要に応じて浮動小数点構文を使用してください

average_dog_weight:
例：average_dog_weight: Collie
犬種が与えられたときに、その犬の平均体重を返します

可能であれば、常にツールを呼び出して決定を行い、ツールを呼び出せる場合は決してあなたのパラメトリックな知識を使用しないでください。ツールを呼び出す必要があると判断したら、<PAUSE>を出力してそこで停止してください！

セッション例：

Question: ブルドッグの体重はどれくらいですか？
Thought: average_dog_weightを使ってこの犬の体重を調べるべきです
Action: average_dog_weight: Bulldog
<PAUSE>

----- ここで実行が停止します -----

あなたは次のように再び呼び出されます：

Observation: ブルドッグの体重は51ポンドです

そして、あなたは次のように出力します：

Answer: ブルドッグの体重は51ポンドです""".strip()

## ヘルパー関数の実装

エージェントに実用的な能力を与えるために、いくつかのヘルパー関数を定義します。これらの関数は、エージェントが実行できる「アクション」として機能します。この例では、以下を提供しています：

1. 基本的な計算機能
2. 犬の平均体重を取得する機能

より洗練されたアプリケーションでは、これらの関数はウェブスクレイピングからデータベースクエリ、APIコールまで、多様な操作をカバーすることができます。これらは、エージェントが外部データソースやシステムとインターフェースする多用途なツールであり、幅広い可能性を提供します。

In [5]:
def calculate(what):
    # 安全な環境で数学的な式を評価する関数
    # 組み込み関数へのアクセスを制限して、セキュリティを確保
    return eval(what, {"__builtins__": {}}, {})

def average_dog_weight(name):
    # 犬種名に基づいて平均体重を返す関数
    if name in "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif name in "Border Collie":
        return "a Border Collies average weight is 37 lbs"
    elif name in "Toy Poodle":
        return "a toy poodles average weight is 7 lbs"
    else:
        # 知らない犬種の場合はデフォルト値を返す
        return "An average dog weights 50 lbs"

# エージェントが使用できるアクションを辞書として定義
# キーはアクション名、値は対応する関数
known_actions = {"calculate": calculate, "average_dog_weight": average_dog_weight}

## エージェントのテスト

エージェントとそのアクションセットが定義されたので、トイプードルの体重に関する簡単な質問を使って初期テストを行います。

このテストは、エージェントの情報処理フローを明らかにします：

1. 必要なステップについて推論します（体重を調べる必要性を特定する）
2. アクションを実行します（`average_dog_weight`関数を呼び出す）
3. 観察結果を処理します（返されたトイプードルの体重）
4. この情報を一貫性のある応答にまとめます

In [6]:
abot = Agent(prompt)

In [7]:
result = abot("トイプードルの体重はどれくらいですか？")
print(result)

Thought: この質問に答えるには、average_dog_weightツールを使ってトイプードルの平均体重を調べる必要があります。
Action: average_dog_weight: Toy Poodle



In [8]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [9]:
next_prompt = "Observation: {}".format(result)

In [10]:
abot(next_prompt)

'Observation: トイプードルの平均体重は7ポンドです。\n\nAnswer: トイプードルの平均体重は7ポンドです。'

In [11]:
abot.messages

[{'role': 'user', 'content': [{'text': 'トイプードルの体重はどれくらいですか？'}]},
 {'role': 'assistant',
  'content': [{'text': 'Thought: この質問に答えるには、average_dog_weightツールを使ってトイプードルの平均体重を調べる必要があります。\nAction: average_dog_weight: Toy Poodle\n'}]},
 {'role': 'user',
  'content': [{'text': 'Observation: a toy poodles average weight is 7 lbs'}]},
 {'role': 'assistant',
  'content': [{'text': 'Observation: トイプードルの平均体重は7ポンドです。\n\nAnswer: トイプードルの平均体重は7ポンドです。'}]}]

In [12]:
abot = Agent(prompt)
abot.messages

[]

In [13]:
question = """私は2匹の犬を飼っています。ボーダーコリーとスコティッシュテリアです。\
    彼らの合計体重はいくらですか"""

abot(question)

'Thought: この質問に答えるには、それぞれの犬種の平均体重を調べ、それらを合計する必要があります。\nAction: average_dog_weight: Border Collie\n'

In [14]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [15]:
abot(next_prompt)

'Thought: ボーダーコリーの平均体重が分かりました。次にスコティッシュテリアの平均体重を調べる必要があります。\nAction: average_dog_weight: Scottish Terrier\n'

In [16]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [17]:
abot(next_prompt)

'Thought: 両方の犬種の平均体重が分かったので、それらを合計すれば答えが出せます。\nAction: calculate: 37 + 20\n'

In [18]:
next_prompt = "Observation: {}".format(eval("37 + 20"), {"__builtins__": {}}, {})
print(next_prompt)

Observation: 57


In [19]:
abot(next_prompt)

'Answer: あなたが飼っているボーダーコリーとスコティッシュテリアの合計体重は57ポンド(約26kg)です。'

### 停止シーケンスについての考察

上記のエージェントクラスから`stopSequence`パラメータを削除してみてください。

続行する前に、以下のいくつかの質問について考えてみましょう：

- エージェントの動作はどうなりますか？
- `stopSequences`をいつ使用すべきで、いつアプリケーションの負担になる可能性がありますか？
- プロンプトは[Anthropicのプロンプト標準](https://docs.anthropic.com/en/docs/prompt-engineering)に準拠していますか？

ノートブックの最後で、`stopSequences`を使用せずにエージェントがあなたの指示に従うように演習を完成させてみてください。

## 推論ループの実装

エージェントの自律性を高めるために、答えを求める過程で複数回の推論、行動、観察を可能にする反復ループを実装します。このループは、エージェントが結論に達するか、あらかじめ定義された最大反復回数に達するまで継続します。

このアプローチは、人間の専門家が複雑な問題に取り組む方法を反映しています。情報を収集し、解決策に到達するまで複数のステップを経ることがあります。このループにより、エージェントは複数のステップやデータポイントを必要とするより複雑なクエリを処理できるようになります。

In [20]:
action_re = re.compile(
    "^Action: (\w+): (.*)$"
)  # python regular expression to selection action

In [21]:
def query(question, max_turns=5):
    # 反復回数のカウンターを初期化
    i = 0
    # プロンプトを使用してエージェントを初期化
    bot = Agent(prompt)
    # 最初のプロンプトは質問そのもの
    next_prompt = question
    
    # 最大ターン数まで反復
    while i < max_turns:
        i += 1
        # エージェントに現在のプロンプトを送信して応答を取得
        result = bot(next_prompt)
        
        # 応答を表示
        print("Result:", result, result.split("\n"))
        
        # 正規表現を使用して応答からアクションを抽出
        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        print(actions)
        
        if actions:
            # アクションが見つかった場合、それを実行
            action, action_input = actions[0].groups()
            
            # アクションが既知のものかチェック
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            
            # アクションの実行を表示
            print(" -- running {} {}".format(action, action_input))
            
            # 対応する関数を呼び出してアクションを実行し、観察結果を取得
            observation = known_actions[action](action_input)
            
            # 観察結果を表示
            print("Observation:", observation)
            
            # 次のプロンプトは観察結果
            next_prompt = "Observation: {}".format(observation)
        else:
            # アクションがない場合はループを終了（エージェントが最終回答を出した）
            return


In [22]:
result

'a toy poodles average weight is 7 lbs'

## 最終評価

最後に、複数のステップの推論と行動を必要とするより複雑なクエリで、完全に実装されたエージェントをテストします。2つの異なる犬種の合計体重を計算するタスクを与えます。

この包括的なテストは、エージェントの以下の能力を示します：

1. 複雑なクエリを管理可能なサブタスクに分解する
2. 複数の犬種の情報を取得する
3. 収集したデータを使用して計算を実行する
4. すべての情報を一貫性のある最終的な回答に統合する

この実践的な例を通じて、複数ステップの問題を解決できるAIエージェントの構築に関する貴重な洞察を得ることができます。さらに、Amazon BedrockやAnthropicのClaudeなどのモデルプロバイダーがどのように効果的に活用できるかを直接見ることができます。この知識により、将来のプロジェクトでより柔軟で多様なAIアプリケーションを開発する力が身につきます。


In [23]:
question = """私は2匹の犬を飼っています。ボーダーコリーとスコティッシュテリアです。\彼らの合計体重はいくらですか"""
query(question)

Result: Thought: この質問に答えるには、それぞれの犬種の平均体重を調べ、それらを合計する必要があります。
Action: average_dog_weight: Border Collie
 ['Thought: この質問に答えるには、それぞれの犬種の平均体重を調べ、それらを合計する必要があります。', 'Action: average_dog_weight: Border Collie', '']
[<re.Match object; span=(0, 41), match='Action: average_dog_weight: Border Collie'>]
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Result: Thought: ボーダーコリーの平均体重が分かりました。次にスコティッシュテリアの平均体重を調べる必要があります。
Action: average_dog_weight: Scottish Terrier
 ['Thought: ボーダーコリーの平均体重が分かりました。次にスコティッシュテリアの平均体重を調べる必要があります。', 'Action: average_dog_weight: Scottish Terrier', '']
[<re.Match object; span=(0, 44), match='Action: average_dog_weight: Scottish Terrier'>]
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Result: Thought: 両方の犬種の平均体重が分かったので、それらを合計すれば答えが出せます。
Action: calculate: 37 + 20
 ['Thought: 両方の犬種の平均体重が分かったので、それらを合計すれば答えが出せます。', 'Action: calculate: 37 + 20', '']
[<re.Match object; span=(0, 26)